In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import spiceypy
import sympy as sym
import math
import numpy.linalg as la
import scipy
import random
from scipy import linalg
from scipy import optimize
#The following code will be made with respect to the Sun as the origin and observations for Mercury

In [2]:
def jd2et(jd):
    """
    Converts Julian Date to JPL NAIF SPICE Ephemeris time.
    Only valid for TDB timescales.
    Parameters:
    -----------
    JD ... Modified Julian Day
    Returns:
    --------
    ET  ... Ephemeris time (ephemeris seconds beyond epoch J2000)
    """
    et = (jd-2451545.0)*86400
    return et

In [3]:
#Specify number of observations (user input)
n = 5

In [4]:
#Obtain x,y,z coordinates and convert to angle pairs
t0 = 2451545.000000000
et = []
for i in range(n):
    et.append(t0+i)
mu = 132712440041.93938
#Initial keplerian state values
kepinit = np.array([5.790906829452708E+07, 2.056302933705567E-01, math.radians(7.005014303275355E+00), math.radians(4.833053855197922E+01), math.radians(2.912428165737838E+01), math.radians(1.747958829169579E+02)])

In [5]:
def MtoE(e, M):
    """
    Converts mean anomaly to eccentric anomaly
    Parameters:
    -----------
    Mean anomaly, eccentricity
    Returns:
    --------
    Eccentric anomaly
    """
    def f(E):
        return E - e*math.sin(E) - M
    def fprime(E):
        return 1 - e*math.cos(E)
    root = scipy.optimize.newton(f, x0=1, fprime=fprime)
    return root

In [6]:
def kep2anglepairs(kep, t):
    """
    Converts keplerian element array to x,y,z coordinates and alpha, delta anglepairs
    Parameters:
    -----------
    Keplerian element array, time
    Returns:
    --------
    x,y,z coordinates and alpha, delta anglepairs
    """
    if (kep.ndim == 2):
        kep = kep[:,0]
    q = kep[0]*(1-kep[1])
    kep[5] = kep[5] - kep[1]*math.sin(kep[5])
    elts = np.hstack((q,kep[1:],jd2et(t),mu))
    xyz = []
    anglepairs = []
    for e in et:
        coordinates = spiceypy.conics(elts, jd2et(e))
        x0 = coordinates[0]
        y0 = coordinates[1]
        z0 = coordinates[2]
        alpha = math.atan(y0 / x0)
        delta = math.asin(z0 / np.sqrt(x0**2 + y0**2 + z0**2))
        xyz.append([x0, y0, z0])
        anglepairs.append([alpha, delta])
    return xyz, anglepairs

In [7]:
#Generate the O matrix using the initial C matrix and random numbers
xyz, C = kep2anglepairs(kepinit, et[0])
C = np.array(C)
C.resize(2*n, 1)
#print(C)

In [8]:
O = C
for o in O:
    o += random.random() / (10**random.randint(8, 10))
    #print(o)
#print(O)

In [9]:
#Partial derivative conversions for keplerian elements
def dalphada(x, y, a, e, I, omegaup, omegalow, E):
    dalphdx = -y / (x**2 + y**2)
    dxda = -math.sin(E)*(np.sqrt(1 - e**2))*(math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I)) + (math.cos(E) - e)*(math.cos(omegaup)*math.cos(omegalow) - math.sin(omegaup)*math.sin(omegalow)*math.cos(I))
    return dalphdx*dxda

def dalphade(x, y, a, e, I, omegaup, omegalow, E):
    dalphdx = -y / (x**2 + y**2)
    dxde = a*(-math.cos(omegaup)*math.cos(omegalow) + math.sin(omegaup)*math.sin(omegalow)*math.cos(I) + math.sin(E)*(e / np.sqrt(1 - e**2))*(math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I)))
    return dalphdx*dxde

def dalphadI(x, y, a, e, I, omegaup, omegalow, E):
    dalphdx = -y / (x**2 + y**2)
    dxdI = a*math.sin(omegaup)*math.sin(omegalow)*((np.sqrt(1 - e**2)) * math.cos(omegalow) * math.sin(E) + math.sin(omegalow)*(math.cos(E) - e))
    return dalphdx*dxdI

def dalphadomegaup(x, y, a, e, I, omegaup, omegalow, E):
    dalphdx = -y / (x**2 + y**2)
    dxdomegaup = a*((math.sin(omegaup)*math.sin(omegalow) - math.cos(omegaup)*math.cos(omegalow)*math.cos(I))*math.sin(E)*np.sqrt(1 - e**2) - (math.cos(E) - e)*(math.sin(omegaup)*math.cos(omegalow) + math.cos(omegaup)*math.sin(omegalow)*math.cos(I)))
    return dalphdx*dxdomegaup

def dalphadomegalow(x, y, a, e, I, omegaup, omegalow, E):
    dalphdx = -y / (x**2 + y**2)
    dxdomegalow = a*((-math.cos(omegaup)*math.cos(omegalow) + math.sin(omegaup)*math.sin(omegalow)*math.cos(I))*math.sin(E)*(np.sqrt(1 - e**2)) - (math.cos(E) - e) * (math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I)))
    return dalphdx*dxdomegalow

def dalphadE(x, y, a, e, I, omegaup, omegalow, E):
    dalphdx = -y / (x**2 + y**2)
    dxdE = -a*((math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I))*math.cos(E)*(np.sqrt(1 - e**2)) + math.sin(E)*(math.cos(omegaup)*math.cos(omegalow) - math.sin(omegaup)*math.sin(omegalow)*math.cos(I)))
    return dalphdx*dxdE

def ddeltada(x, y, z, a, e, I, omegaup, omegalow, E):
    ddeltdx = -(x*z) / ((np.sqrt(x**2 + y**2))*(x**2 + y**2 + z**2))
    dxda = -math.sin(E)*(np.sqrt(1 - e**2))*(math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I)) + (math.cos(E) - e)*(math.cos(omegaup)*math.cos(omegalow) - math.sin(omegaup)*math.sin(omegalow)*math.cos(I))
    return ddeltdx*dxda

def ddeltade(x, y, z, a, e, I, omegaup, omegalow, E):
    ddeltdx = -(x*z) / ((np.sqrt(x**2 + y**2))*(x**2 + y**2 + z**2))
    dxde = a*(-math.cos(omegaup)*math.cos(omegalow) + math.sin(omegaup)*math.sin(omegalow)*math.cos(I) + math.sin(E)*(e / np.sqrt(1 - e**2))*(math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I)))
    return ddeltdx*dxde

def ddeltadI(x, y, z, a, e, I, omegaup, omegalow, E):
    ddeltdx = -(x*z) / ((np.sqrt(x**2 + y**2))*(x**2 + y**2 + z**2))
    dxdI = a*math.sin(omegaup)*math.sin(omegalow)*((np.sqrt(1 - e**2)) * math.cos(omegalow) * math.sin(E) + math.sin(omegalow)*(math.cos(E) - e))
    return ddeltdx*dxdI

def ddeltadomegaup(x, y, z, a, e, I, omegaup, omegalow, E):
    ddeltdx = -(x*z) / ((np.sqrt(x**2 + y**2))*(x**2 + y**2 + z**2))
    dxdomegaup = a*((math.sin(omegaup)*math.sin(omegalow) - math.cos(omegaup)*math.cos(omegalow)*math.cos(I))*math.sin(E)*np.sqrt(1 - e**2) - (math.cos(E) - e)*(math.sin(omegaup)*math.cos(omegalow) + math.cos(omegaup)*math.sin(omegalow)*math.cos(I)))
    return ddeltdx*dxdomegaup

def ddeltadomegalow(x, y, z, a, e, I, omegaup, omegalow, E):
    ddeltdx = -(x*z) / ((np.sqrt(x**2 + y**2))*(x**2 + y**2 + z**2))
    dxdomegalow = a*((-math.cos(omegaup)*math.cos(omegalow) + math.sin(omegaup)*math.sin(omegalow)*math.cos(I))*math.sin(E)*(np.sqrt(1 - e**2)) - (math.cos(E) - e) * (math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I)))
    return ddeltdx*dxdomegalow

def ddeltadE(x, y, z, a, e, I, omegaup, omegalow, E):
    ddeltdx = -(x*z) / ((np.sqrt(x**2 + y**2))*(x**2 + y**2 + z**2))
    dxdE = -a*((math.cos(omegaup)*math.sin(omegalow) + math.sin(omegaup)*math.cos(omegalow)*math.cos(I))*math.cos(E)*(np.sqrt(1 - e**2)) + math.sin(E)*(math.cos(omegaup)*math.cos(omegalow) - math.sin(omegaup)*math.sin(omegalow)*math.cos(I)))
    return ddeltdx*dxdE

In [10]:
def get_A(xyzcoord, elts):
    """
    Gets A matrix from x,y,z coordinates and keplerian element array
    Parameters:
    -----------
    x,y,z coordinates, keplerian element array
    Returns:
    --------
    A matrix
    """
    c = 0
    #E = MtoE(elts[1], elts[5])
    E = elts[5]
    for i in range(0, 2*n, 2):
        A[i][0] = dalphada(xyzcoord[c][0], xyzcoord[c][1], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i][1] = dalphade(xyzcoord[c][0], xyzcoord[c][1], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i][2] = dalphadI(xyzcoord[c][0], xyzcoord[c][1], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i][3] = dalphadomegaup(xyzcoord[c][0], xyzcoord[c][1], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i][4] = dalphadomegalow(xyzcoord[c][0], xyzcoord[c][1], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i][5] = dalphadE(xyzcoord[c][0], xyzcoord[c][1], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i+1][0] = ddeltada(xyzcoord[c][0], xyzcoord[c][1], xyzcoord[c][2], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i+1][1] = ddeltade(xyzcoord[c][0], xyzcoord[c][1], xyzcoord[c][2], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i+1][2] = ddeltadI(xyzcoord[c][0], xyzcoord[c][1], xyzcoord[c][2], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i+1][3] = ddeltadomegaup(xyzcoord[c][0], xyzcoord[c][1], xyzcoord[c][2], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i+1][4] = ddeltadomegalow(xyzcoord[c][0], xyzcoord[c][1], xyzcoord[c][2], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        A[i+1][5] = ddeltadE(xyzcoord[c][0], xyzcoord[c][1], xyzcoord[c][2], elts[0], elts[1], elts[2], elts[3], elts[4], E)
        c+=1
    return A

In [11]:
x = kepinit[:, None].copy()
#print(x.shape)
x[5] = MtoE(x[1], x[5])
A = np.zeros((2*n, 6))

#Iterative least-square process to find final keplerian state array
for i in range(5):
    xyz, C = kep2anglepairs(x, et[i])
    C = np.array(C)
    C.resize(2*n, 1)
    A = get_A(xyz, x)
    b = O - C
    #print(A)
    #print(b)
    x += np.linalg.lstsq(A, b)[0]
    #print(x)

<ipython-input-11-ae0d5296458d>:15: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  x += np.linalg.lstsq(A, b)[0]


In [12]:
xfinal = x

In [13]:
print(xfinal)

[[5.79090683e+07]
 [1.95261746e-01]
 [1.04593604e-01]
 [9.44135062e-01]
 [6.08618869e-01]
 [3.02351866e+00]]


In [14]:
print(kepinit)

[5.79090683e+07 2.05630293e-01 1.22260564e-01 8.43527027e-01
 5.08314607e-01 3.03211216e+00]


In [16]:
xfinal[:,0] - kepinit

array([ 0.        , -0.01036855, -0.01766696,  0.10060803,  0.10030426,
       -0.0085935 ])